#Import lib

In [ ]:
! pip install opendatasets

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#Load Dataset

In [ ]:
od.download("https://www.kaggle.com/datasets/ahsanaseer/top-rated-tmdb-movies-10k?fbclid=IwAR2MpWrWpcw2QNCv_FZg2l0sjBh9xAvhrqtnZBO9K-QS6PHI1aHkdB6qLa0")

100%|██████████| 1.43M/1.43M [00:00<00:00, 2.62MB/s]

In [ ]:
movies=pd.read_csv('top-rated-tmdb-movies-10k/top10K-TMDB-movies.csv')
movies.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


#Preprocessing Data

In [ ]:
movies.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
movies.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [ ]:
movies=movies[['id','title','overview','genre']]
movies.head()

,id,title,overview,genre
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime"
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance"
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime"
3,424,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War"
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...,"Drama,Crime"


In [ ]:
movies['tags'] = movies['overview']+movies['genre']

In [ ]:
data  = movies.drop(columns=['overview', 'genre'])

In [ ]:
cv=CountVectorizer(max_features=10000, stop_words='english')

In [ ]:
vector=cv.fit_transform(data['tags'].values.astype('U')).toarray()
vector.shape

(10000, 10000)

#Apply similarity

In [ ]:
similarity=cosine_similarity(vector)
similarity

array([[1.        , 0.05634362, 0.12888482, ..., 0.07559289, 0.11065667,
        0.06388766],
       [0.05634362, 1.        , 0.07624929, ..., 0.        , 0.03636965,
        0.        ],
       [0.12888482, 0.07624929, 1.        , ..., 0.02273314, 0.06655583,
        0.08645856],
       ...,
       [0.07559289, 0.        , 0.02273314, ..., 1.        , 0.03253   ,
        0.02817181],
       [0.11065667, 0.03636965, 0.06655583, ..., 0.03253   , 1.        ,
        0.0412393 ],
       [0.06388766, 0.        , 0.08645856, ..., 0.02817181, 0.0412393 ,
        1.        ]])

In [ ]:
data[data['title']=="The Godfather"].index[0]

2

In [ ]:
distance = sorted(list(enumerate(similarity[2])), reverse=True, key=lambda vector:vector[1])
for i in distance[1:4]:
    print(data.iloc[i[0]].title)

The Godfather: Part II
Blood Ties
Joker


In [ ]:
def recommand(movies):
    index=data[data['title']==movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[1:4]:
        print(data.iloc[i[0]].title)

#Test

In [ ]:
recommand("The Godfather")

The Godfather: Part II
Blood Ties
Joker


In [ ]:
recommand("The Dark Knight")

Batman: The Long Halloween, Part Two
Batman: The Long Halloween, Part One
The Dark Knight Rises


In [ ]:
recommand("12 Angry Men")

12 Angry Men
Conviction
Selena
